# Process

## Import libraries

In [1]:
from viapy.api import ViafAPI, ViafEntity, SRUResult, SRUItem
import pandas as pd
import pickle as pkl
import time

## Read files

In [2]:
books_with_results = pd.read_csv("data/items_books_after_viaf_and_filling.csv", low_memory = False, index_col=0).fillna("").drop(["year", "publisher"], axis=1)

In [3]:
all_books = pd.read_csv("data/items_books_after_viaf.csv", low_memory = False, index_col=0).fillna("").drop("viaf_id", axis = 1).drop(["year", "publisher"], axis=1)

In [4]:
results = pd.merge(all_books.reset_index(), books_with_results, on = list(all_books.columns), how="left").set_index("index").fillna("")

In [5]:
results

,ISBN,title,author,alt_title,alt_author,correct_author,alt_first_author,viaf_id
index,,,,,,,,
0,0195153448,Classical Mythology,Mark P. O. Morford,Classical Mythology,Mark P. O. Morford|Robert J. Lenardon,Mark P. O. Morford,Mark P. O. Morford,92220559
1,0801319536,Classical Mythology,Mark P. O. Morford,Classical Mythology,Mark P. O. Morford|Robert J. Lenardon,Mark P. O. Morford,Mark P. O. Morford,92220559
2,0002005018,Clara Callan,Richard Bruce Wright,Clara Callan,Richard Bruce Wright,Richard Bruce Wright,Richard Bruce Wright,64022406
3,0771597185,The teacher's daughter,Richard Bruce Wright,The Teacher's Daughter,Richard Bruce Wright,Richard Bruce Wright,Richard Bruce Wright,64022406
4,0060973129,Decision in Normandy,Carlo D'Este,Decision in Normandy,Carlo D'Este,Carlo D'Este,Carlo D'Este,97755576
...,...,...,...,...,...,...,...,...
269269,0762412119,"Burpee Gardening Cyclopedia: A Concise, Up to ...",Allan Armitage,Burpee Gardening Cyclopedia,Maureen Heffernan|Holly H. Shimizu|Allan Armit...,Allan Armitage,Maureen Heffernan,61645853
269270,1582380805,Tropical Rainforests: 230 Species in Full Colo...,"Allen M., Ph.D. Young",Tropical Rainforests,Allen M. Young,Allen M. Young,Allen M. Young,25946864
269271,1845170423,Cocktail Classics,David Biggs,Cocktail Classics,David Biggs,David Biggs,David Biggs,121661213


In [33]:
results.to_csv("data/items_books_after_viaf_and_filling_and_merging.csv")

## So far results

In [5]:
print("With the work so far, we found viaf id for", len(results[results.viaf_id!=""]), "books.")

With the work so far, we found viaf id for 242607 books.


In [6]:
print("With the work so far, we found same author through Google Books for", len(results[results.correct_author!=""]), "books.")

With the work so far, we found same author through Google Books for 247457 books.


In [7]:
print("With the work so far, we found any match through Google Books for", len(results[results.alt_author!=""]), "books.")

With the work so far, we found any match through Google Books for 259177 books.


In [8]:
results[results.viaf_id==""].loc[results.correct_author!=""].drop_duplicates("correct_author")

,ISBN,title,author,alt_title,alt_author,correct_author,alt_first_author,viaf_id
index,,,,,,,,
529,0061076031,Mary-Kate &amp; Ashley Switching Goals (Mary-K...,Mary-Kate &amp; Ashley Olsen,Mary-Kate & Ashley Switching Goals,Mary-Kate & Ashley Olsen,Mary-Kate & Ashley Olsen,Mary-Kate & Ashley Olsen,
542,0061066605,Two of a Kind #20: Sealed with a Kiss (Two of ...,Mary-Kate &amp; Ashley Olsen,Two of a Kind #20: Sealed with a Kiss,Mary-kate & Ashley Olsen,Mary-kate & Ashley Olsen,Mary-kate & Ashley Olsen,
9596,0062770500,Seattle Access,HarperReference,Seattle Access,HarperReference|J. Kingston Pierce,HarperReference,HarperReference,
11955,0060973897,Lakota Woman,Dog Mary Crow,Lakota Woman,Dog Mary Crow,Dog Mary Crow,Dog Mary Crow,
15500,0590870157,My Garden (Beginning Literacy),Jesse S. Ostrow,My Garden,Jesse S. Ostrow,Jesse S. Ostrow,Jesse S. Ostrow,
...,...,...,...,...,...,...,...,...
269183,0806962518,Rubber Stamp Celebrations: Dazzling Projects f...,Personal Stamp Exchange,Rubber Stamp Celebrations,Personal Stamp Exchange (U.S.),Personal Stamp Exchange (U.S.),Personal Stamp Exchange (U.S.),
269202,0913337064,"\B\"" Movie Trivia Quiz""",Walter Haan,"""B"" Movie Trivia Quiz",Walter Haan,Walter Haan,Walter Haan,
269210,0877830592,Quetico Wolf,Robert B. Oetting,Quetico Wolf,Robert B. Oetting|Rae Oetting,Robert B. Oetting,Robert B. Oetting,


# Continue the work

In [9]:
def get_viaf_entry(author_name):
    print(author_name)
    viaf = ViafAPI()
    viaf_id = ""
    potential_entries = viaf.suggest(author_name)[:5]
    if len(potential_entries)==0:
        print("No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    for entry in potential_entries:
        if entry["nametype"]=="personal":
            print(entry["term"])
            try:
                viaf_id = entry["viafid"]
                print("We found viaf_id!")
            except:
                print(author_name + " has no Viaf ID.")
            break
        print("ISSUE! We didn't find for", author_name)
    return viaf_id

In [6]:
unfilled_results = results[results.viaf_id==""].loc[results.correct_author==""].drop_duplicates(["author","alt_first_author"])

In [7]:
unfilled_results

,ISBN,title,author,alt_title,alt_author,correct_author,alt_first_author,viaf_id
index,,,,,,,,
22,039592684X,The Best American Short Stories 1999 (Best Ame...,Amy Tan,The Best American Short Stories 1999,Katrina Kenison,,Katrina Kenison,
25,1931056323,The Bonesetter's Daughter,Amy Tan,,,,,
63,0399145761,What If?: The World's Foremost Military Histor...,Robert Cowley,What If?,Stephen E. Ambrose|David McCullough|John Keega...,,Stephen E. Ambrose,
64,0671870432,PLEADING GUILTY,Scott Turow,,,,,
86,0671519166,GUILTY AS CHARGED,Scott Turow,Guilty as Charged,Mystery Writers of America,,Mystery Writers of America,
...,...,...,...,...,...,...,...,...
269181,0838934854,Anglo-American Cataloguing Rules,Michael Gorman,Anglo-American Cataloguing Rules,,,,
269191,0380894920,Trivial Conquest: The Smart Reference Source f...,"Lisa &amp; Frankel, Eric Merkin",Trivial Conquest,Lisa Merkin|Eric Frankel,,Lisa Merkin,
269214,3150013763,Der Hofmeister,Jakob Lenz,Der Hofmeister oder Vorteile der Privaterziehung,Jakob Michael Reinhold Lenz|Karl Siegfried Guthke,,Jakob Michael Reinhold Lenz,


In [25]:
start = time.time()
i=0
for index, row in unfilled_results.iloc[2000:].iterrows():
    if results.at[index,"viaf_id"]=="":
        author = row.author
        amount_of_words = len(author.split(" "))
        if amount_of_words>1:
            viaf_id = get_viaf_entry(author)

            if (viaf_id == "") & (row.alt_author!=""):
                print("try Google Books first author name")
                viaf_id = get_viaf_entry(row.alt_first_author)
        elif (row.alt_author!=""):
            print("IT WAS ONLY ONE WORD!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!",author)
            viaf_id = get_viaf_entry(row.alt_first_author)
        results.at[index, "viaf_id"] = viaf_id
        if i%500==0:
            print(index, time.time()-start)
            results.to_csv("data/items_books_after_viaf_and_filling_and_merging_and_more_viaf.csv")
        i+=1

People Magazine
ISSUE! We didn't find for People Magazine
64297 0.5577881336212158
Kelli M. Gary
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
Mary J. Latis
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Kelli M. Gary
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
Ground Zero (Project)
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Kelli M. Gary
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Lee D. Willoughby
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Anatolian Treasures
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Faculty of Theology of the University of
ISSUE! We didn't find for Faculty of Theology of the University of
ISSUE! We didn't find for Faculty of Theology of the University of
ISSUE! We didn't find for Faculty of Theology of the University of
ISSUE! We didn't find for Faculty of Theology of the University of
try Google Books first author name
Universid

No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Not Applicable (Na )
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
Leon Amiel Publisher
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Not Applicable (Na )
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
Smithsonian Institution
ISSUE! We didn't find for Smithsonian Institution
ISSUE! We didn't find for Smithsonian Institution
ISSUE! We didn't find for Smithsonian Institution
ISSUE! We didn't find for Smithsonian Institution
ISSUE! We didn't find for Smithsonian Institution
Not Applicable (Na )
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
Paperblank Book Company Staff
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Not Applicable (Na )
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
American Bible Society
ISSUE! We didn't find for American Bible Society
I

No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
Rand McNally
ISSUE! We didn't find for Rand McNally
ISSUE! We didn't find for Rand McNally
ISSUE! We didn't find for Rand McNally
ISSUE! We didn't find for Rand McNally
ISSUE! We didn't find for Rand McNally
Not Applicable (Na )
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
Andrews McMeel Publishing
ISSUE! We didn't find for Andrews McMeel Publishing
ISSUE! We didn't find for Andrews McMeel Publishing
ISSUE! We didn't find for Andrews McMeel Publishing
ISSUE! We didn't find for Andrews McMeel Publishing
ISSUE! We didn't find for Andrews McMeel Publishing
Not Applicable (Na )
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
Wishing Well
ISSUE! We didn't find for Wishing Well
ISSUE! We didn't find for Wishing Well
ISSUE! We didn't find for Wishing Well
ISSUE! We didn't find for Wishing Well
Not Applicable (Na )
No resu

No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Not Applicable (Na )
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
Knorr, Inc
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Not Applicable (Na )
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
Council on Resident Education in Obstetrics and Gynecology. Committee on Educational Objectives
ISSUE! We didn't find for Council on Resident Education in Obstetrics and Gynecology. Committee on Educational Objectives
Not Applicable (Na )
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
Fodor's Travel Publications, Inc. Staff
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Not Applicable (Na )
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
Pearson Education Australia Staff
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Not Applicable (Na )
No results!!!!!!!!!!

No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Jarrold Publishing
ISSUE! We didn't find for Jarrold Publishing
ISSUE! We didn't find for Jarrold Publishing
ISSUE! We didn't find for Jarrold Publishing
ISSUE! We didn't find for Jarrold Publishing
ERIC MD BERNE
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Kenneth Uptegrove
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Earma Broadway Brown
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Jonathan A. Michaels
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
Livingstone Editorial Group
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Tanya A. Crosby
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Time Life
ISSUE! We didn't find for Time Life
ISSUE! We didn't find for Time Life
ISSUE! We didn't find for Time Life
ISSUE! We didn't find for Time Life
ISSUE! We didn't find for Time Life
Time Life Books
ISSUE! We didn't find for Time Life Books
ISSUE! We didn't find 

...

ISSUE! We didn't find for Prevention Magazine Health Books
ISSUE! We didn't find for Prevention Magazine Health Books
Nancy Topping Bazin
Nancy Topping Bazin, 1934-
We found viaf_id!
Joachim Baschin
Joachim Baschin
We found viaf_id!
J. A. Guy
J. A Guy
We found viaf_id!
J.B. Davis MBA CFP
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
Jackson B. Davis
Jackson B Davis
We found viaf_id!
IT WAS ONLY ONE WORD!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Cazotte
Jacques Cazotte
Jacques Cazotte, 1719-1792
We found viaf_id!
COLE M &amp; J
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
Michael Cole
Michael Cole, 1938-
We found viaf_id!
S. Abel
S Abele
We found viaf_id!
Jennie Mackenzie
Jennie Mackenzie
We found viaf_id!
Thomas E. Griess
Thomas E Griess
We found viaf_id!
Laura Secord
Laura Secord, 1775-1868
We found viaf_id!
Stephen Gillers
Stephen Gillers, 1943-
We found viaf_id!
USS Benfold, Captain D Abrasho

No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Bowden Family
Bowden family
We found viaf_id!
Schulz M.
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
Charles M. Schulz
Charles M. Schulz, 1922-2000
We found viaf_id!
Allison Pirot
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
Alison Lohans
Alison Lohans, 1949-
We found viaf_id!
IT WAS ONLY ONE WORD!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Bayless
Maureen Bayless
Maureen Bayless, 1959-
We found viaf_id!
Marshall Mandell
Marshall Mandell
We found viaf_id!
Kingston Medland
Kingston Medland
We found viaf_id!
Catherine Buckaway
Catherine Buckaway, 1919-
We found viaf_id!
L. J Middleton
L. J.  Middleton, 1938-
We found viaf_id!
Kathleen Pitt
Kathleen Pitt, 1952-
We found viaf_id!
Robert M., Jr Appleton
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
Appleton
Appleton, Lawrence, 1890-1937
We found viaf_id!
R

No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Lucien Soulban
Lucien Soulban
We found viaf_id!
Diane Simone Dixon
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
Diane Simone Vezza
Diane Simone Vezza
We found viaf_id!
Christian R. Hirsch
Christian R. von Hirschhausen 1964-
We found viaf_id!
Kathy Pohl
Kathy Pohl
We found viaf_id!
Ethan Casey (editor) BlueEar.com
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
John Sheppard
John Sheppard, 1515?-1558
We found viaf_id!
David Hope
David Hope, Baron Hope of Craighead
We found viaf_id!
American Cancer Society
ISSUE! We didn't find for American Cancer Society
ISSUE! We didn't find for American Cancer Society
ISSUE! We didn't find for American Cancer Society
ISSUE! We didn't find for American Cancer Society
ISSUE! We didn't find for American Cancer Society
Lynette Bassman
Lynette Bassman, 1959-
We found viaf_id!
IT WAS ONLY ONE WORD!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! INTL
Amer

No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
David W. Sifton
David W Sifton
We found viaf_id!
Eric Klein
Eric Klein
We found viaf_id!
Harper San Francisco Staff
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
try Google Books first author name
HarperCollins Publishers
ISSUE! We didn't find for HarperCollins Publishers
ISSUE! We didn't find for HarperCollins Publishers
ISSUE! We didn't find for HarperCollins Publishers
ISSUE! We didn't find for HarperCollins Publishers
ISSUE! We didn't find for HarperCollins Publishers
Ben, Dr Bova
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Lindley Boegehold
Lindley Boegehold
We found viaf_id!
Lisa A. Hooper
Lisa A Hooper
We found viaf_id!
Inc Junior League of Augusta
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
J.A. Kjelgaard
J. A. Kjelgaard, 1910-1959
We found viaf_id!
Sandra Gookin
Sandra Gookin
We found viaf_id!
Angie Sage
Angie Sage, 1952-
We found viaf_id!
Dk Pub
ISSUE! We didn'

No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Menas Kafatos
Menas Kafatos
We found viaf_id!
N. Thompson
N. Thompson Hobbs
We found viaf_id!
Russell Mitten
Russell Mitten
We found viaf_id!
Carol Lasser
Carol Lasser
We found viaf_id!
Alec Finlay
Alec Finlay
We found viaf_id!
Brady Joan
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Elinor R. Workman
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
John Cygan
John Cygan American actor
We found viaf_id!
IT WAS ONLY ONE WORD!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Harrison&amp;
Henry Harrison
Henry Harrison
We found viaf_id!
Owen Jones
Owen Jones, 1809-1874
We found viaf_id!
Johannes Willem Bertens
Johannes Willem Bertens
We found viaf_id!
IT WAS ONLY ONE WORD!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Duras
Marguerite Duras
Marguerite Duras, 1914-1996
We found viaf_id!
Neil F. Michelsen
Neil-F Michelsen
We found viaf_id!
Glenn Hunsucker
No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
IT WAS ONLY ONE WOR

In [24]:
unfilled_results

,ISBN,title,author,alt_title,alt_author,correct_author,alt_first_author,viaf_id
index,,,,,,,,
22,039592684X,The Best American Short Stories 1999 (Best Ame...,Amy Tan,The Best American Short Stories 1999,Katrina Kenison,,Katrina Kenison,
25,1931056323,The Bonesetter's Daughter,Amy Tan,,,,,
63,0399145761,What If?: The World's Foremost Military Histor...,Robert Cowley,What If?,Stephen E. Ambrose|David McCullough|John Keega...,,Stephen E. Ambrose,
64,0671870432,PLEADING GUILTY,Scott Turow,,,,,
86,0671519166,GUILTY AS CHARGED,Scott Turow,Guilty as Charged,Mystery Writers of America,,Mystery Writers of America,
...,...,...,...,...,...,...,...,...
269181,0838934854,Anglo-American Cataloguing Rules,Michael Gorman,Anglo-American Cataloguing Rules,,,,
269191,0380894920,Trivial Conquest: The Smart Reference Source f...,"Lisa &amp; Frankel, Eric Merkin",Trivial Conquest,Lisa Merkin|Eric Frankel,,Lisa Merkin,
269214,3150013763,Der Hofmeister,Jakob Lenz,Der Hofmeister oder Vorteile der Privaterziehung,Jakob Michael Reinhold Lenz|Karl Siegfried Guthke,,Jakob Michael Reinhold Lenz,


In [22]:
len(unfilled_results)

15542

In [27]:
results.to_csv("data/items_books_after_viaf_and_filling_and_merging_and_more_viaf.csv")

In [26]:
results[results.viaf_id==""]

,ISBN,title,author,alt_title,alt_author,correct_author,alt_first_author,viaf_id
index,,,,,,,,
39,0399141154,Hundred Secret Senses Limited Ed.,Amy Tan,,,,,
40,1590400356,The Kitchen God's Wife,Amy Tan,,,,,
41,1931056331,The Bonesetter's Daughter,Amy Tan,,,,,
49,078710566X,The Hundred Secret Senses,Amy Tan,,,,,
51,B0001FZGPI,The Bonesetter's Daughter,Amy Tan,,,,,
...,...,...,...,...,...,...,...,...
269183,0806962518,Rubber Stamp Celebrations: Dazzling Projects f...,Personal Stamp Exchange,Rubber Stamp Celebrations,Personal Stamp Exchange (U.S.),Personal Stamp Exchange (U.S.),Personal Stamp Exchange (U.S.),
269202,0913337064,"\B\"" Movie Trivia Quiz""",Walter Haan,"""B"" Movie Trivia Quiz",Walter Haan,Walter Haan,Walter Haan,
269210,0877830592,Quetico Wolf,Robert B. Oetting,Quetico Wolf,Robert B. Oetting|Rae Oetting,Robert B. Oetting,Robert B. Oetting,


In [19]:
len(results)

269274